In [57]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [58]:
mldf = pd.read_csv('../EDA/final.csv')
mldf.dtypes


Unnamed: 0                   int64
state_fips                 float64
state_name                  object
fips                         int64
county_name_x               object
mask_mandate                 int64
month_started                int64
mandate_duration_months      int64
date                        object
county                      object
state                       object
cases                        int64
deaths                       int64
popestimate2020            float64
lat                        float64
lng                        float64
e_results                    int64
cases_perc                   int64
deaths_perc                  int64
binned                       int64
dtype: object

In [59]:
mldf_numerical = mldf.select_dtypes(include = 'number')
mldf_numerical.dtypes

Unnamed: 0                   int64
state_fips                 float64
fips                         int64
mask_mandate                 int64
month_started                int64
mandate_duration_months      int64
cases                        int64
deaths                       int64
popestimate2020            float64
lat                        float64
lng                        float64
e_results                    int64
cases_perc                   int64
deaths_perc                  int64
binned                       int64
dtype: object

In [60]:
#drop columns that don't need to be input features
mldf_for_scaling = mldf_numerical.drop(columns=['Unnamed: 0', 'state_fips', 'fips', 'month_started', 'cases', 'deaths', 'popestimate2020'])


In [61]:
mldf_for_scaling.dtypes

mask_mandate                 int64
mandate_duration_months      int64
lat                        float64
lng                        float64
e_results                    int64
cases_perc                   int64
deaths_perc                  int64
binned                       int64
dtype: object

In [62]:
# assign variables, y = depedent variable - death percentage; x = independent variable - mask_mandate
y = mldf_for_scaling['cases_perc'].values
#y[:5]
X = mldf_for_scaling.copy()
X = X.drop('cases_perc', axis=1)

In [63]:
#scale the new dataset
scaler = StandardScaler()
mldf_scaled = scaler.fit_transform(mldf_for_scaling)

In [64]:
mldf_scaled[:5]

array([[ 1.04855251,  0.75202069, -0.69578892,  0.48939295,  0.75906895,
         1.17128652, -0.5292253 ,  0.43846785],
       [ 1.04855251,  0.75202069, -0.81970784,  0.33872072,  0.75906895,
         0.10573641,  0.25870354,  0.43846785],
       [ 1.04855251,  1.08682368, -0.93194611,  0.4177963 ,  0.75906895,
         0.46091978, -0.07898025,  0.43846785],
       [ 1.04855251,  0.75202069, -1.06710234,  0.33352458,  0.75906895,
         0.46091978,  1.49687745,  0.43846785],
       [ 1.04855251,  0.75202069, -0.69895002,  0.44542926,  0.75906895,
        -0.24944696, -0.75434783,  0.43846785]])

In [65]:
print(np.mean(mldf_scaled[:,0]))
print(np.std(mldf_scaled[:,0]))


-1.4556573331833038e-16
1.0


In [66]:
mldf_scaled.shape

(3124, 8)

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [68]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=78)
classifier

LogisticRegression(random_state=78)

In [69]:
LogisticRegression(C=1.0, class_weight='binned', dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=5000, multi_class='warn', penalty='12',
   random_state=78, solver='lbfgs', tol=0.0001, warm_start=False)

LogisticRegression(class_weight='binned', max_iter=5000, multi_class='warn',
                   penalty='12', random_state=78)

In [70]:
classifier.fit(X_train, y_train)

/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=78)

In [71]:
#Create predictions and assemble the results into a dataframe

predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,7,10
1,4,5
2,2,2
3,5,3
4,6,8
...,...,...
776,6,6
777,5,8
778,7,6
779,5,5


In [72]:
# Assess performance

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.2560819462227913

In [73]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        15
           2       0.33      0.15      0.20        34
           3       0.22      0.32      0.26        38
           4       0.30      0.34      0.32        82
           5       0.43      0.45      0.44        94
           6       0.29      0.42      0.34       125
           7       0.20      0.36      0.26       104
           8       0.14      0.09      0.11       106
           9       0.20      0.18      0.19        77
          10       0.00      0.00      0.00        44
          11       0.00      0.00      0.00        20
          12       0.00      0.00      0.00        14
          13       0.00      0.00      0.00        11
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         4
          17       0.00    

/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Deaths Prediction

In [74]:
# assign variables, y = depedent variable - death percentage; x = independent variable - mask_mandate
y1 = mldf_for_scaling['deaths_perc'].values
#y[:5]
X1 = mldf_for_scaling.copy()
X1 = X.drop('deaths_perc', axis=1)

In [75]:
#scale the new dataset
scaler = StandardScaler()
mldf_scaled_deaths = scaler.fit_transform(mldf_for_scaling)

In [76]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=78)

In [77]:
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression(solver='lbfgs', random_state=78)
classifier1

LogisticRegression(random_state=78)

In [78]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=1500, multi_class='warn', penalty='12',
   random_state=78, solver='lbfgs', tol=0.0001, warm_start=False)

LogisticRegression(max_iter=1500, multi_class='warn', penalty='12',
                   random_state=78)

In [79]:
classifier1.fit(X1_train, y1_train)

/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=78)

In [80]:
predictions_deaths = classifier1.predict(X1_test)
pd.DataFrame({"Prediction": predictions_deaths, "Actual": y1_test})

,Prediction,Actual
0,9,8
1,5,4
2,5,0
3,5,3
4,9,3
...,...,...
776,4,12
777,7,10
778,9,7
779,4,13


In [81]:
# Assess performance

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions_deaths)

0.18053777208706787

## Random Forest 

In [82]:
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators = 1000, random_state = 1)

In [83]:
rf_model = rf_model.fit(X_train, y_train)

In [84]:
from sklearn.metrics import balanced_accuracy_score
rf_y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, rf_y_pred)

/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.2210107921936987

In [85]:
rf_cm = confusion_matrix(y_test, rf_y_pred )
rf_cm

array([[ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  9,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 6, 10,  9,  4,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  0,  0],
       [ 9,  2,  5, 17,  2,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1],
       [ 6,  0,  7, 16, 11, 38,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  3],
       [ 1,  2,  7, 15, 15, 50,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1],
       [ 4,  4,  2,  7,  0,  4, 53,  2,  1,  4, 31,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  9,  2,  1],
       [ 0,  2,  3,  2,  0,  0, 37,  2,  2, 10, 36,  0,  0,  0,  0,  0,
         0,  1,  1,  0,  0,  4,  1,  3],
       [ 2,  0,  0,  3,  0,  0, 34,  3,  3,  8, 47,  0,  0,  0,  0,  1,
         0,  0,  3,  1, 

In [86]:
classification_report(y_test, rf_y_pred)

/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/mlenv/lib/

'              precision    recall  f1-score   support\n\n           0       0.03      1.00      0.05         1\n           1       0.30      0.60      0.40        15\n           2       0.26      0.26      0.26        34\n           3       0.26      0.45      0.33        38\n           4       0.35      0.13      0.19        82\n           5       0.53      0.53      0.53        94\n           6       0.36      0.42      0.39       125\n           7       0.22      0.02      0.04       104\n           8       0.27      0.03      0.05       106\n           9       0.20      0.08      0.11        77\n          10       0.15      0.61      0.24        44\n          11       0.00      0.00      0.00        20\n          12       0.00      0.00      0.00        14\n          13       0.00      0.00      0.00        11\n          14       0.00      0.00      0.00         2\n          15       0.00      0.00      0.00         4\n          16       0.00      0.00      0.00         4\n       